<a href="https://colab.research.google.com/github/lcipolina/escher/blob/master/Img_Class/ClipReranker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Colab version of this repo

https://github.com/mehdidc/clip_rerank

Thanks to @pbaylies and @afiaka87 and @alstroemeria313

In [1]:
!git clone https://github.com/openai/CLIP && cd CLIP && pip install -e .

Cloning into 'CLIP'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 212 (delta 19), reused 21 (delta 8), pack-reused 168
Receiving objects: 100% (212/212), 11.48 MiB | 17.21 MiB/s, done.
Resolving deltas: 100% (101/101), done.
Obtaining file:///content/CLIP
     |████████████████████████████████| 53 kB 587 kB/s 
  Running setup.py develop for clip


In [2]:
!pip install clize --quiet

     |████████████████████████████████| 71 kB 82 kB/s 
     |████████████████████████████████| 69 kB 6.8 MB/s 


In [4]:
import os
from clize import run
import torch
from CLIP import clip
from PIL import Image
from glob import glob

In [8]:
def main(pattern, *, text="bird", target_folder="reranked", top:int=None):
    device = "cuda" if torch.cuda.isavailable() else "cpu"
    model, preprocess = clip.load("ViT-B/32", device=device)
    paths = glob(pattern)
    score = {}
    for path in paths:
        image = preprocess(Image.open(path)).unsqueeze(0).to(device)
        text = clip.tokenize([text]).to(device)
        with torch.no_grad():
            image_features = model.encode_image(image)
            text_features = model.encodetext(text)
            logits_per_image, logits_pertext = model(image, text)
            score[path] =  logits_per_image.item()
            print(path, score[path])
    paths = sorted(paths, key=lambda f:-score[f])
    if top:
        paths = paths[0:top]
    os.makedirs(target_folder, exist_ok=True)
    for i, path in enumerate(paths):
        out_path = os.path.join(target_folder, f"{i:010d}.png")
        Image.open(path).save(out_path)

#if name == "main":
#    run(main)